# ML linear regression

In this assignment, we will predict the price of a house by its region, size, number of bedrooms, etc.

## Introduction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression

import urllib.request 

import pytest
import ipytest
import unittest

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer

ipytest.autoconfig()

df = pd.read_csv("../../assets/data/housing.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

Features | Informations
--- | --- |
`longitude` | A measure of how far west a house is; a higher value is farther west
`latitude` | A measure of how far north a house is; a higher value is farther north
`housing_median_age` | Median age of a house within a block; a lower number is a newer building
`total_rooms` | Total number of rooms within a block
`total_bedrooms` | Total number of bedrooms within a block
`population` | Total number of people residing within a block
`households` | Total number of households, a group of people residing within a home unit, for a block
`median_income` | Median income for households within a block of houses (measured in tens of thousands of US Dollars)
`median_house_value` | Median house value for households within a block (measured in US Dollars)
`oceanProximity` | Location of the house w.r.t ocean/sea

Source: Kaggle

In [ ]:
df.info()

In [ ]:
len(df)

In [ ]:
len(df.columns)

So, we have 20640 data points and 10 features. In those 10 features, 9 features are input features and the feature `median_house_value` is the target variable/label.

## Task

### Task 1: Exploratory data analysis

#### Split the data set into a training set and a test set

In [ ]:
train_data, test_data = train_test_split(cal_data, test_size=0.1,random_state=20)

#### Checking data statistics

In [ ]:
train_data.describe(include='all').transpose()

#### Checking missing values

In [ ]:
train_data.isnull().sum()

In [ ]:
print('The Percentage of missing values in total_bedrooms is: {}%'.format(train_data.isnull().sum()['total_bedrooms'] / len(train_data) * 100))

#### Checking values in the categorical feature(s)

In [ ]:
train_data['ocean_proximity'].value_counts()

In [ ]:
sns.countplot(data=train_data, x='ocean_proximity')

#### Checking Correlation Between Features

In [ ]:
correlation = train_data.corr()
correlation['median_house_value']

In [ ]:
plt.figure(figsize=(12,7))

sns.heatmap(correlation,annot=True,cmap='crest')

Some features like total_bedrooms and households are highly correlated. Same things for `total_bedrooms` and `total_rooms` and that makes sense because for many houses, the number of people who stay in that particular house (`households`) goes with the number of available rooms(`total_rooms`) and `bed_rooms`.

The other interesting insights is that the `price of the house` is closely correlated with the `median income`, and that makes sense too. For many cases, you will resonably seek house that you will be able to afford based on your income.

#### Plotting geographical features

Since we have latitude and longitude, let's plot it. It can help us to know the location of certain houses on the map and hopefully this will resemble California map.

In [ ]:
plt.figure(figsize=(12,7))

sns.scatterplot(data = train_data, x='longitude', y='latitude')

In [ ]:
plt.figure(figsize=(12,7))

sns.scatterplot(data = train_data, x='longitude', y='latitude', hue='median_house_value')

It makes sense that the most expensive houses are those close to sea. We can verify that with the `ocean_proximity`.

In [ ]:
plt.figure(figsize=(12,7))

sns.scatterplot(data = train_data, x='longitude', y='latitude', hue='ocean_proximity', 
                size='median_house_value')

Yup, all houses near the ocean are very expensive compared to other areas.

#### Exploring Relationship Between Individual Features

In [ ]:
plt.figure(figsize=(12,7))

sns.scatterplot(data = train_data, x='median_house_value', y='median_income', hue='housing_median_age')

There are times you want to quickly see different plots to draw insights from the data. In that case, you can use grid plots. Seaborn, a visualization library provides a handy function for that.

In [ ]:
sns.pairplot(train_data)

As you can see, it plots the relationship between all numerical features and histograms of each feature as well. But it's slow...

To summarize the data exploration, the goal here it to understand the data as much as you can. There is no limit to what you can inspect. And understanding the data will help you build an effective ML systems.

### Task 3: Data preprocessing

#### Create the input data and output data for training the machine learning model

Since we are going to prepare the data for the ML model, let's create an input training data and the training label, label being `median_house_value`.

In [ ]:
training_input_data = train_data.drop('median_house_value', axis=1)
training_labels = train_data['median_house_value']

In [ ]:
training_input_data.head()

In [ ]:
training_labels.head()

#### Handling missing values

In this example, we will fill the values with the mean of the concerned features.

In [ ]:
# We are going to impute all numerical features
# Ideally, we would only impute bed_rooms because it's the one possessing NaNs
num_feats = training_input_data.drop('ocean_proximity', axis=1)

def handle_missing_values(input_data):
  """
  Docstring 

  # This is a function to take numerical features...
  ...and impute the missing values
  # We are filling missing values with mean
  # fit_transform fit the imputer on input data and transform it immediately
  # You can use fit(input_data) and then transform(input_data) or
  # Or do it at once with fit.transform(input_data)
  # Imputer returns the imputed data as a NumPy array 
  # We will convert it back to Pandas dataframe

  """
  mean_imputer = SimpleImputer(strategy='mean')
  num_feats_imputed = mean_imputer.fit_transform(input_data)
  num_feats_imputed = pd.DataFrame(num_feats_imputed, 
                            columns=input_data.columns, index=input_data.index )


  return num_feats_imputed

In [ ]:
num_feats_imputed = handle_missing_values(num_feats)
num_feats_imputed.isnull().sum()

The feature `total_bedroom` was the one having missing values. Looking above, we no longer have the missing values in whole dataframe.

#### Encoding categorical features

Categorical features are features which have categorical values. An example in our dataset is `ocean_proximity` that has the following values.

In [ ]:
training_input_data['ocean_proximity'].value_counts()

So we have 5 categories: `<1H OCEAN`, `INLAND`, `NEAR OCEAN`, `NEAR BAY`, `ISLAND`.

##### Mapping

Mapping is simple. We create a dictionary of categorical values and their corresponding numerics. And after that, we map it to the categorical feature.

In [ ]:
cat_feats = training_input_data['ocean_proximity']
cat_feats.value_counts()

In [ ]:
feat_map = {
      '<1H OCEAN': 0,
      'INLAND': 1,
      'NEAR OCEAN': 2,
      'NEAR BAY': 3, 
      'ISLAND': 4
}

cat_feats_encoded = cat_feats.map(feat_map)

In [ ]:
cat_feats_encoded.head()

Cool, all categories were mapped to their corresponding numerals. That is actually encoding. We are converting the categories (in text) into numbers, typically because ML models expect numeric inputs.

##### Handling categorical features with Sklearn

Sklearn has many preprocessing functions to handle categorical features. Ordinary Encoder is one of them. It does the same as what we did before with mapping. The only difference is implementation.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

def ordinary_encoder(input_data):
  
  encoder = OrdinalEncoder()
  
  output = encoder.fit_transform(input_data)

  return output

In [ ]:
cat_feats_enc = ordinary_encoder(cat_feats)
cat_feats_enc

##### One hot encoding

One hot encoding is most preferred when the categories are not in any order and that is exactly how our categorical feature is. This is what I mean by saying unordered categories: If you have 3 cities and encode them with numbers (1,2,3) respectively, a machine learning model may learn that city 1 is close to city 2 and to city 3. As that is a false assumption to make, the model will likely give incorrect predictions if the city feature plays an important role in the analysis.

On the flip side, if you have the feature of ordered ranges like low, medium, and high, then numbers can be an effective way because you want to keep the sequence of these ranges.

In our case, the ocean proximity feature is not in any order. By using one hot, The categories will be converted into binary representation (1s or 0s), and the orginal categorical feature will be splitted into more features, equivalent to the number of categories.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def one_hot(input_data):

  one_hot_encoder = OneHotEncoder()
  output = one_hot_encoder.fit_transform(input_data)
  
  # The output of one hot encoder is a sparse matrix. 
  # It's best to convert it into numpy array 
  output = output.toarray()

  return output

In [ ]:
cat_feats = training_input_data[['ocean_proximity']]

cat_feats_hot = one_hot(cat_feats)

cat_feats_hot

Cool, we now have one hot matrix, where categories are represented in 1s or 0s. As one hot create more additional features, if you have a categorical feature having many categories, it can be too much, hence resulting in poor performance.

#### Scaling numerical 

Most machine learning models will work well when given small input values, and best if they are in the same range. For that reason, there are two most techniques to scale features:

* Normalization where the features are scaled to values between 0 and 1. And
* Standardization where the features are rescaled to have 0 mean and unit standard deviation. When working with datasets containing outliers (such as time series), standardization is the right option in that particular case.

In [ ]:
## Normalizing numerical features 

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

num_scaled = scaler.fit_transform(num_feats)
num_scaled

In [ ]:
## Standardizing numerical features 

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

num_scaled = scaler.fit_transform(num_feats)
num_scaled

#### Putting all data preprocessing steps into a single pipeline

We are going to do three things:

* Creating a numerical pipeline having all numerical preprocessing steps (handling missing values and standardization)
* Creating a categorical pipeline to encode the categorical features
* Combining both pipelines into one pipeline.

##### Creating a numerical features pipeline

In [ ]:
from sklearn.pipeline import Pipeline

num_feats_pipe = Pipeline([
                  ('imputer', SimpleImputer(strategy='mean')), 
                  ('scaler', StandardScaler())         
            ])

num_feats_preprocessed = num_feats_pipe.fit_transform(num_feats)

num_feats_preprocessed

In [ ]:
num_feats_pipe.steps[0]

In [ ]:
num_feats_pipe.steps[1]

##### Pipeline for transforming categorical features

In [ ]:
cat_feats_pipe = Pipeline([
     ('encoder', OneHotEncoder())                      
])

cat_feats_preprocessed = cat_feats_pipe.fit_transform(cat_feats)

In [ ]:
type(cat_feats_preprocessed)

##### Final data processing pipeline

In [ ]:
from sklearn.compose import ColumnTransformer

# The transformer requires lists of features

num_list = list(num_feats)
cat_list = list(cat_feats)

final_pipe = ColumnTransformer([
   ('num', num_feats_pipe, num_list),    
   ('cat', cat_feats_pipe, cat_list)                        

])

training_data_preprocessed = final_pipe.fit_transform(training_input_data)

In [ ]:
training_data_preprocessed

In [ ]:
type(training_data_preprocessed)

### Choosing and training a model

In [ ]:
from sklearn.linear_model import LinearRegression

reg_model = LinearRegression()

In [ ]:
reg_model.fit(training_data_preprocessed, training_labels)

Great, that was fast! The model is now trained on the training set.

Before we evaluate the model, let's take things little deep.

Have you heard of things called weights and bias? These are two paremeters of any typical ML model. It is possible to access the model paremeters, here is how.

In [ ]:
# Coef or coefficients are referred to as weights

reg_model.coef_

In [ ]:
# Intercept is what can be compared to the bias 

reg_model.intercept_

### Model evaluation

In [ ]:
from sklearn.metrics import mean_squared_error

predictions = reg_model.predict(training_data_preprocessed)

In [ ]:
mse = mean_squared_error(training_labels, predictions)

rmse = np.sqrt(mse)
rmse 

In [ ]:
train_data.describe().median_house_value['mean']

#### Model evaluation with cross validation

In [ ]:
from sklearn.model_selection import cross_val_score

scoring = 'neg_root_mean_squared_error' 

scores = cross_val_score(reg_model, training_data_preprocessed, training_labels, scoring=scoring, cv=10)

In [ ]:
scores = -scores

scores.mean()

You can also use `cross_val_predict` to make a prediction on the training and validation subsets.

In [ ]:
from sklearn.model_selection import cross_val_predict

predictions = cross_val_predict(reg_model, training_data_preprocessed, training_labels, cv=10)

In [ ]:
mse_cross_val = mean_squared_error(training_labels, predictions)
rmse_cross_val = np.sqrt(mse_cross_val)
rmse_cross_val 

To evaluate the model on the test set, we will have to preprocess the test dat as we preprocessed the training data. This is a general rule for all machine learning models. The test input data must be in the same format as the data that the model was trained on.

In [ ]:

test_input_data = test_data.drop('median_house_value', axis=1)
test_labels = test_data['median_house_value']


test_preprocessed = final_pipe.transform(test_input_data)

In [ ]:
test_pred = reg_model.predict(test_preprocessed)
test_mse = mean_squared_error(test_labels,test_pred)

test_rmse = np.sqrt(test_mse)
test_rmse

## Acknowledgments
Thanks to Nyandwi for creating the open-source course [Linear Models for regression](https://github.com/Nyandwi/machine_learning_complete/blob/main/6_classical_machine_learning_with_scikit-learn/1_linear_models_for_regression.ipynb). It inspires the majority of the content in this chapter.